In [121]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from dotenv import load_dotenv
import os


In [122]:
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)
load_dotenv()
browser.maximize_window()
senha = os.getenv("SENHA")
wait = WebDriverWait(browser, 3)

browser.get(
    "https://sipe.dpu.def.br/sip/login.php?sigla_orgao_sistema=DPU&sigla_sistema=SEI")

In [123]:
browser.find_element(By.ID, "txtUsuario").send_keys("maykon.soares")
browser.find_element(By.ID, "pwdSenha").send_keys(senha)
Select(browser.find_element(By.ID, "selOrgao")).select_by_value("8038")
browser.find_element(By.ID, "Acessar").click()
element = wait.until(EC.element_to_be_clickable(
    (By.CLASS_NAME, "sparkling-modal-close")))

element.click()
browser.find_element(By.CLASS_NAME, "ancoraPadraoPreta").click()


In [124]:

elements = browser.find_elements(By.CLASS_NAME, "processoVisualizado")
tags = browser.find_element(By.CLASS_NAME, "imagemStatus")

for element, tag in zip(elements, tags):
    element.send_keys(Keys.CONTROL + Keys.RETURN)
    janelas = browser.window_handles
    browser.switch_to.window(janelas[1]) 

    ActionChains(browser)\
        .send_keys(Keys.ARROW_DOWN)\
        .send_keys(Keys.ARROW_DOWN)\
        .send_keys(Keys.ARROW_DOWN)\
        .send_keys(Keys.ARROW_DOWN)\
        .perform()

    iframe = wait.until(EC.presence_of_element_located((By.ID, "ifrVisualizacao")))
    browser.switch_to.frame(iframe)

    iframe = wait.until(EC.presence_of_element_located((By.ID, "ifrArvoreHtml")))
    browser.switch_to.frame(iframe)


    value = browser.find_element(By.CLASS_NAME, "Texto_Justificado_Recuo_Primeira_Linha")
    text_content = value.text

    parts = text_content.split(", ", 1)

    if len(parts) > 1:
        result = parts[1]
    else:
        result = ""
        
        
    print(result)

    browser.switch_to.default_content()

    browser.find_element(By.ID, "lnkControleProcessos").click()
    tag.click()

    browser.find_element(By.CLASS_NAME, "imagemStatus").click()


    ActionChains(browser)\
        .send_keys(result)\
        .send_keys(Keys.ENTER)\
        .perform()

    browser.find_element(By.NAME, "sbmRegistrarAnotacao").click()
    browser.close()
    browser.switch_to.window(janelas[0]) 





TypeError: 'WebElement' object is not iterable